In [1]:
import os

import zipfile
import pandas as pd

import re

import xml.etree.ElementTree as ET

from tqdm.notebook import tqdm as tq

import pickle

In [157]:
year = '2010'

In [158]:
directory_in_str = r"/media/seb/data/EPfulltext_backfile/%s" %year
directory_in_str

'/media/seb/data/EPfulltext_backfile/2010'

In [159]:
files = []
for branch in os.walk(directory_in_str):
    dirpath, _, filenames = branch
    # print(dirpath, _, filenames)
    for file in filenames:
        if '.zip' in file and 'B1' in file:
            # print(dirpath, file)
            files.append(dirpath+'/'+file)

In [160]:
len(files)

58116

In [161]:
def get_claims(root):
    all_claims = {'en':[], 'de':[], 'fr':[]}
    for child in root:
        if child.tag == 'claims':
            lang = child.attrib['lang']
            texts = []
            for i in child:
                text = ''
                for e in i.findall('.//'):
                    try:
                        text += e.text
                    except:
                        pass
                texts.append(text)
            all_claims[lang] = texts
    return all_claims['en'], all_claims['de'], all_claims['fr']

In [162]:
def get_ipcr_cls(root):
    ipcrs = []
    for e in root.findall('.//classification-ipcr'):
        for _ in e.findall('.//'):
            ipcr = _.text
            ipcr = re.sub('\s{1,}', ' ', ipcr).strip()
            ipcrs.append(ipcr)
    return ipcrs

In [163]:
def get_titles(root):
    text = ''
    for e in root.findall('.//B540'):
        for _ in e.findall('.//'):
            text += _.text + '\t'
    return text

In [164]:
all_file_details = []
for file in tqdm(files):
    # print('Checking file:', file, end='\r')
    archive = zipfile.ZipFile(file, 'r')
    for i in archive.filelist:
        filename = i.filename
        if 'B1.xml' in filename:
            file_details = {}
            df = archive.read(filename)
            tree = ET.ElementTree(ET.fromstring(df))
            root = tree.getroot()
            en_claims, de_claims, fr_claims = get_claims(root)
            ipcrs = get_ipcr_cls(root)
            titles = get_titles(root)
            
            file_details['name'] = filename
            file_details['ipcrs'] = ipcrs
            file_details['titles'] = titles
            file_details['en_claims'] = en_claims
            file_details['de_claims'] = de_claims
            file_details['fr_claims'] = fr_claims           
            all_file_details.append(file_details)

100%|██████████| 58116/58116 [08:03<00:00, 120.20it/s]


In [166]:
details_df = pd.DataFrame(all_file_details)

In [167]:
details_df

,name,ipcrs,titles,en_claims,de_claims,fr_claims
0,EP03768234NWB1.xml,"[G11B 7/24 20060101AFI20040901BHEP, G11B 7/26 ...",de\tOPTISCHES AUFZEICHNUNGSMEDIUM; STANZER ZUR...,[An optical recording medium (1) having groove...,"[Ein optisches Speichermedium (1) mit Rillen, ...",[Support d'enregistrement optique (1) possédan...
1,EP05708307NWB1.xml,"[B42D 15/00 20060101AFI20050907BHEP, G07D 7/20...",de\tVORRICHTUNG UND VERFAHREN ZUR IDENTIFIZIER...,[A method of verifying that an object (10) is ...,"[Ein Verfahren zum Verifizieren, dass ein Obje...",[Une méthode pour vérifier qu'un objet (10) es...
2,EP06741313NWB1.xml,[B01D 53/26 20060101AFI20070510BHEP],de\tVERBESSERTE VORRICHTUNG ZUR KÜHLTROCKNUNG\...,[Device for cool drying which comprises a heat...,"[Vorrichtung zur Kühltrocknung, welche einen W...","[Dispositif pour le séchage par réfrigération,..."
3,EP00100245NWB1.xml,"[H04Q 11/04 20060101AFI20000515BHEP, H04J 3/16...",de\tVorrichtung unter Verwendung eines In-Band...,"[A transmission apparatus, comprising a plural...","[Übertragungsvorrichtung, umfassend eine Mehrz...","[Appareil de transmission, comprenant une plur..."
4,EP00101367NWB1.xml,[G06F 17/30 20060101AFI20000512BHEP],de\tMultimodaler Informationzugriff\ten\tMulti...,[A method for information browsing using preco...,[Ein Verfahren zum Browsen von Informationen u...,[Procédé de navigation à travers des informati...
...,...,...,...,...,...,...
58111,EP99945525NWB1.xml,"[A61K 38/48 20060101AFI20060329BHEP, A61P 31/0...",de\tMETHODE ZUR BEHANDLUNG VON STAPHYLOKOKKENI...,[Use of a lysostaphin analogue for the manufac...,[Verwendung eines Lysostaphin-Analogons für di...,[Utilisation d'un analogue de lysostaphine pou...
58112,EP99946542NWB1.xml,[H02M 1/088 20060101AFI20000314BHEP],de\tKOPPELUNG FUER AKTIVE HALBLEITERELEMENTE\t...,[A method of coupling in series active semicon...,[Verfahren zum Koppeln in Reihenschaltung von ...,[Procédé de couplage en série de composants se...
58113,EP99951136NWB1.xml,[F16H 61/18 20060101AFI20000515BHEP],de\tREGELEINRICHTUNG FÜR AUTOMATISCHE ÜBERTRAG...,[A control device for an automatic transmissio...,[Steuervorrichtung für ein Automatikgetriebe (...,[Dispositif de contrôle destiné à une transmis...
58114,EP99952693NWB1.xml,"[C07K 14/47 20060101AFI20010508BHEP, C07K 16/1...",de\tVERFAHREN ZUR HERSTELLUNG DER MONOMEREN B...,[A method of making a monomeric β-form of a pr...,[Verfahren zur Herstellung einer monomeren ß-F...,[Procédé de fabrication d'une forme β monomère...


In [168]:
fname = 'epo_%s_df.gzip' %year
fname

'epo_2010_df.gzip'

In [169]:
%%time
details_df.to_pickle(fname, compression='gzip')

CPU times: user 31.1 s, sys: 483 ms, total: 31.5 s
Wall time: 31.6 s


## Load dataframe

In [170]:
# year = 2020
fname = 'epo_%s_df.gzip' %year
fname

'epo_2010_df.gzip'

In [171]:
%%time
details_df = pd.read_pickle(fname, compression='gzip')

CPU times: user 3.35 s, sys: 265 ms, total: 3.61 s
Wall time: 3.62 s


In [172]:
details_df

,name,ipcrs,titles,en_claims,de_claims,fr_claims
0,EP03768234NWB1.xml,"[G11B 7/24 20060101AFI20040901BHEP, G11B 7/26 ...",de\tOPTISCHES AUFZEICHNUNGSMEDIUM; STANZER ZUR...,[An optical recording medium (1) having groove...,"[Ein optisches Speichermedium (1) mit Rillen, ...",[Support d'enregistrement optique (1) possédan...
1,EP05708307NWB1.xml,"[B42D 15/00 20060101AFI20050907BHEP, G07D 7/20...",de\tVORRICHTUNG UND VERFAHREN ZUR IDENTIFIZIER...,[A method of verifying that an object (10) is ...,"[Ein Verfahren zum Verifizieren, dass ein Obje...",[Une méthode pour vérifier qu'un objet (10) es...
2,EP06741313NWB1.xml,[B01D 53/26 20060101AFI20070510BHEP],de\tVERBESSERTE VORRICHTUNG ZUR KÜHLTROCKNUNG\...,[Device for cool drying which comprises a heat...,"[Vorrichtung zur Kühltrocknung, welche einen W...","[Dispositif pour le séchage par réfrigération,..."
3,EP00100245NWB1.xml,"[H04Q 11/04 20060101AFI20000515BHEP, H04J 3/16...",de\tVorrichtung unter Verwendung eines In-Band...,"[A transmission apparatus, comprising a plural...","[Übertragungsvorrichtung, umfassend eine Mehrz...","[Appareil de transmission, comprenant une plur..."
4,EP00101367NWB1.xml,[G06F 17/30 20060101AFI20000512BHEP],de\tMultimodaler Informationzugriff\ten\tMulti...,[A method for information browsing using preco...,[Ein Verfahren zum Browsen von Informationen u...,[Procédé de navigation à travers des informati...
...,...,...,...,...,...,...
58111,EP99945525NWB1.xml,"[A61K 38/48 20060101AFI20060329BHEP, A61P 31/0...",de\tMETHODE ZUR BEHANDLUNG VON STAPHYLOKOKKENI...,[Use of a lysostaphin analogue for the manufac...,[Verwendung eines Lysostaphin-Analogons für di...,[Utilisation d'un analogue de lysostaphine pou...
58112,EP99946542NWB1.xml,[H02M 1/088 20060101AFI20000314BHEP],de\tKOPPELUNG FUER AKTIVE HALBLEITERELEMENTE\t...,[A method of coupling in series active semicon...,[Verfahren zum Koppeln in Reihenschaltung von ...,[Procédé de couplage en série de composants se...
58113,EP99951136NWB1.xml,[F16H 61/18 20060101AFI20000515BHEP],de\tREGELEINRICHTUNG FÜR AUTOMATISCHE ÜBERTRAG...,[A control device for an automatic transmissio...,[Steuervorrichtung für ein Automatikgetriebe (...,[Dispositif de contrôle destiné à une transmis...
58114,EP99952693NWB1.xml,"[C07K 14/47 20060101AFI20010508BHEP, C07K 16/1...",de\tVERFAHREN ZUR HERSTELLUNG DER MONOMEREN B...,[A method of making a monomeric β-form of a pr...,[Verfahren zur Herstellung einer monomeren ß-F...,[Procédé de fabrication d'une forme β monomère...


In [173]:
iprcs = details_df.ipcrs.to_list()
auto_indices = []
for ind, i in enumerate(iprcs):
    for _ in i:
        if 'B60' in _:
            if ind not in auto_indices:
                auto_indices.append(ind)
len(auto_indices)

3627

In [174]:
auto_filtered_df = details_df.loc[auto_indices]
auto_filtered_df = auto_filtered_df.reset_index(drop=True)
auto_filtered_df

,name,ipcrs,titles,en_claims,de_claims,fr_claims
0,EP00989672NWB1.xml,"[B60Q 1/26 20060101AFI20011009BHEP, B60R 1/12 ...",de\tSEITENSPIEGELEINRICHTUNG MIT BELEUCHTUNG U...,[A side mirror assembly (30;70;90;150;190;280)...,"[Seitenspiegelanordnung (30, 70, 90, 150,190, ...",[Ensemble (30 ; 70 ; 90 ; 150 ; 190 ; 280) de ...
1,EP01103618NWB1.xml,[B60Q 1/30 20060101AFI20010614BHEP],de\tBremsleuchte mit integrierter Scheibenwasc...,[Motor vehicle having a brake light which has ...,"[Kraftfahrzeug mit einer Bremsleuchte, die ein...",[Véhicule automobile comprenant un feu de frei...
2,EP01127254NWB1.xml,"[H01M 8/04 20060101AFI20091125BHEP, B60L 11/18...",de\tBrennstoffzellen-Stromversorgungseinheit\t...,[Method for controlling a fuel cell power supp...,[Verfahren zum Steuern/Regeln einer Brennstoff...,[Procédé de commande d'une unité d'alimentatio...
3,EP03255614NWB1.xml,"[B60B 27/00 20060101AFI20091015BHEP, F16C 43/0...",de\tLagervorrichtung für ein Fahrzeugrad\ten\t...,[A bearing apparatus for a driving wheel of ve...,[Lagervorrichtung für ein Antriebsrad eines Fa...,[Dispositif de roulement pour une roue motrice...
4,EP02251985NWB1.xml,"[B60K 31/00 20060101AFI20020805BHEP, B60T 7/22...","de\tAdaptives, abstandsbezogenes Fahrgeschwind...",[An adaptive cruise control system for an auto...,[Adaptives Fahrgeschwindigkeits-Regelsystem fü...,[Régulateur de vitesse adaptatif pour un véhic...
...,...,...,...,...,...,...
3622,EP08789042NWB1.xml,"[B60T 13/58 20060101AFI20090305BHEP, B60W 30/1...","de\tBREMSVORRICHTUNG, BREMSSTEUEREINHEIT UND B...",[A brake apparatus including: a regenerative b...,[Bremsvorrichtung mit: einer Regenerativbremse...,[Appareil de frein comportant :\nune unité (10...
3623,EP08789096NWB1.xml,"[B60K 28/06 20060101AFI20100111BHEP, G01N 33/4...",de\tALCOTESTGERÄT\ten\tBREATHALYZER TEST DEVIC...,[A breathalyzer test device (1) comprising:\nv...,"[Alkoholmesstestgerät (1), aufweisend:\neine W...",[Dispositif d'alcootest (1) comprenant :\nun m...
3624,EP09153247NWB1.xml,"[B60R 13/02 20060101AFI20090513BHEP, B62D 27/0...",de\tVerbindungsmittel zum Verbinden von Einbau...,[Connection assembly (10) for connecting build...,[Verbindungsmittel (10) zum Verbinden von Einb...,[Moyen de liaison (10) pour relier des élément...
3625,EP09163972NWB1.xml,"[B60K 13/04 20060101AFI20091221BHEP, F01N 13/1...",de\tHalterung für das Auspuffsystem eines Fahr...,"[Exhaust support (1) for a vehicle, in particu...","[Fahrzeug-Auspuffträger (1), insbesondere für ...",[Support d'échappement (1) de véhicule notamme...


In [175]:
filt_name = 'epo_%s_b60_df.gzip' %year
filt_name

'epo_2010_b60_df.gzip'

In [176]:
%%time
auto_filtered_df.to_pickle(filt_name, compression='gzip')

CPU times: user 1.56 s, sys: 51.5 ms, total: 1.62 s
Wall time: 1.62 s


In [177]:
for title, en_claims, de_claims in zip(auto_filtered_df.titles[:1],
                                      auto_filtered_df.en_claims[:1],
                                      auto_filtered_df.de_claims[:1]):
    print(title)
    print(len(en_claims))
    print(len(de_claims))
    print()
    print(en_claims[0])
    print()
    print(de_claims[0])

de	SEITENSPIEGELEINRICHTUNG MIT BELEUCHTUNG UND SIGNALLEUCHTEN FÜR FAHRZEUGE	en	VEHICLE SIDE MIRROR ASSEMBLY WITH INTEGRAL ILLUMINATION AND SIGNAL LIGHTING	fr	RETROVISEUR AUTOMOBILE LATERAL AVEC ECLAIRAGE INTEGRAL ET ECLAIRAGE DE SIGNALISATION	
37
37

A side mirror assembly (30;70;90;150;190;280) for a vehicle, comprising:
a mounting member (32;72;92;152;192;282), having a mounting surface (286) for attachment of said mounting member to the vehicle;a reflective element (35) attached to said mounting member at a location spaced from said mounting surface; anda waveguide (42;82;102;162;164;196;248) contained within said mounting member and extending from a first location proximate said mounting surface to a second location 46;104;170;202 proximate said reflective element,characterized in that

Seitenspiegelanordnung (30, 70, 90, 150,190, 280) für ein Fahrzeug, umfassend:
ein Befestigungselement (32, 72, 92, 152, 192, 282) mit einer Befestigungsoberfläche (286) zum Anbringen des Befestigu